In [26]:
import pandas as pd
import numpy as np
import re
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.impute import KNNImputer



#pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)


In [27]:
df_flight = pd.read_csv('../Files/Customer Flight Activity.csv')
df_loyalty = pd.read_csv('../Files/Customer Loyalty History.csv')

In [28]:
df_flight.head()

,Loyalty Number,Year,Month,Flights Booked,Flights with Companions,Total Flights,Distance,Points Accumulated,Points Redeemed,Dollar Cost Points Redeemed
0,100018,2017,1,3,0,3,1521,152.0,0,0
1,100102,2017,1,10,4,14,2030,203.0,0,0
2,100140,2017,1,6,0,6,1200,120.0,0,0
3,100214,2017,1,0,0,0,0,0.0,0,0
4,100272,2017,1,0,0,0,0,0.0,0,0


In [29]:
df_flight.describe().T

,count,mean,std,min,25%,50%,75%,max
Loyalty Number,405624.0,550037.873084,258935.286969,100018.0,326961.00,550834.0,772194.00,999986.0
Year,405624.0,2017.500000,0.500001,2017.0,2017.00,2017.5,2018.00,2018.0
Month,405624.0,6.500000,3.452057,1.0,3.75,6.5,9.25,12.0
Flights Booked,405624.0,4.115052,5.225518,0.0,0.00,1.0,8.00,21.0
Flights with Companions,405624.0,1.031805,2.076869,0.0,0.00,0.0,1.00,11.0
Total Flights,405624.0,5.146858,6.521227,0.0,0.00,1.0,10.00,32.0
Distance,405624.0,1208.880059,1433.155320,0.0,0.00,488.0,2336.00,6293.0
Points Accumulated,405624.0,123.692721,146.599831,0.0,0.00,50.0,239.00,676.5
Points Redeemed,405624.0,30.696872,125.486049,0.0,0.00,0.0,0.00,876.0
Dollar Cost Points Redeemed,405624.0,2.484503,10.150038,0.0,0.00,0.0,0.00,71.0


In [30]:
df_flight.dtypes

Loyalty Number                   int64
Year                             int64
Month                            int64
Flights Booked                   int64
Flights with Companions          int64
Total Flights                    int64
Distance                         int64
Points Accumulated             float64
Points Redeemed                  int64
Dollar Cost Points Redeemed      int64
dtype: object

In [31]:
df_flight.isnull().sum()

Loyalty Number                 0
Year                           0
Month                          0
Flights Booked                 0
Flights with Companions        0
Total Flights                  0
Distance                       0
Points Accumulated             0
Points Redeemed                0
Dollar Cost Points Redeemed    0
dtype: int64

In [32]:
df_flight.duplicated().sum()

1864

In [33]:
cols = ['Country', 'Province', 'City',
       'Gender', 'Education', 'Salary', 'Marital Status', 'Loyalty Card',
       'CLV', 'Enrollment Type', 'Enrollment Year', 'Enrollment Month',
       'Cancellation Year', 'Cancellation Month']

for column in cols:
    print(f"Frequência de valores na coluna '{column}':\n")
    print(df_loyalty[column].value_counts())

Frequência de valores na coluna 'Country':

Country
Canada    16737
Name: count, dtype: int64
Frequência de valores na coluna 'Province':

Province
Ontario                 5404
British Columbia        4409
Quebec                  3300
Alberta                  969
Manitoba                 658
New Brunswick            636
Nova Scotia              518
Saskatchewan             409
Newfoundland             258
Yukon                    110
Prince Edward Island      66
Name: count, dtype: int64
Frequência de valores na coluna 'City':

City
Toronto           3351
Vancouver         2582
Montreal          2059
Winnipeg           658
Whistler           582
Halifax            518
Ottawa             509
Trenton            486
Edmonton           486
Quebec City        485
Dawson Creek       444
Fredericton        425
Regina             409
Kingston           401
Tremblant          398
Victoria           389
Hull               358
West Vancouver     324
St. John's         258
Thunder Bay        256
S

In [34]:
df_loyalty[df_loyalty['Cancellation Year'].notnull()]

,Loyalty Number,Country,Province,City,Postal Code,Gender,Education,Salary,Marital Status,Loyalty Card,CLV,Enrollment Type,Enrollment Year,Enrollment Month,Cancellation Year,Cancellation Month
2,429460,Canada,British Columbia,Vancouver,V6E 3D9,Male,College,NaN,Single,Star,3839.75,Standard,2014,7,2018.0,1.0
19,354730,Canada,New Brunswick,Fredericton,E3B 2H2,Female,College,NaN,Married,Star,3885.46,Standard,2014,10,2018.0,1.0
31,201574,Canada,Ontario,Ottawa,K1F 2R2,Female,Bachelor,51375.0,Married,Star,3893.31,Standard,2015,4,2015.0,12.0
39,834891,Canada,British Columbia,Vancouver,V5R 1W3,Female,Bachelor,54460.0,Married,Star,3911.12,Standard,2017,3,2017.0,4.0
45,733338,Canada,Ontario,Thunder Bay,K8T 5M5,Female,Bachelor,80697.0,Married,Star,3919.37,Standard,2016,6,2017.0,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16706,409984,Canada,Yukon,Whitehorse,Y2K 6R0,Male,College,NaN,Single,Star,7452.23,2018 Promotion,2018,4,2018.0,12.0
16713,969719,Canada,British Columbia,West Vancouver,V6V 8Z3,Male,Bachelor,88356.0,Married,Star,40235.97,Standard,2016,5,2017.0,1.0
16718,795743,Canada,Ontario,Thunder Bay,K8T 5M5,Male,College,NaN,Married,Star,43019.01,Standard,2013,11,2014.0,7.0
16730,672917,Canada,Ontario,Toronto,M1R 4K3,Female,Bachelor,76778.0,Single,Star,58166.55,Standard,2015,1,2015.0,9.0


In [35]:
df_loyalty.describe().T

,count,mean,std,min,25%,50%,75%,max
Loyalty Number,16737.0,549735.880445,258912.132453,100018.00,326603.00,550434.00,772019.00,999986.00
Salary,12499.0,79245.609409,35008.297285,-58486.00,59246.50,73455.00,88517.50,407228.00
CLV,16737.0,7988.896536,6860.982280,1898.01,3980.84,5780.18,8940.58,83325.38
Enrollment Year,16737.0,2015.253211,1.979111,2012.00,2014.00,2015.00,2017.00,2018.00
Enrollment Month,16737.0,6.669116,3.398958,1.00,4.00,7.00,10.00,12.00
Cancellation Year,2067.0,2016.503145,1.380743,2013.00,2016.00,2017.00,2018.00,2018.00
Cancellation Month,2067.0,6.962748,3.455297,1.00,4.00,7.00,10.00,12.00


In [36]:
df_loyalty[df_loyalty['Salary'] <0]['Salary'].count() # ADDING COUNT TO SEE HOW MANY ARE AFTER SORTING


20

In [37]:
df_loyalty.duplicated().sum()

0

In [38]:
df_loyalty.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16737 entries, 0 to 16736
Data columns (total 16 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Loyalty Number      16737 non-null  int64  
 1   Country             16737 non-null  object 
 2   Province            16737 non-null  object 
 3   City                16737 non-null  object 
 4   Postal Code         16737 non-null  object 
 5   Gender              16737 non-null  object 
 6   Education           16737 non-null  object 
 7   Salary              12499 non-null  float64
 8   Marital Status      16737 non-null  object 
 9   Loyalty Card        16737 non-null  object 
 10  CLV                 16737 non-null  float64
 11  Enrollment Type     16737 non-null  object 
 12  Enrollment Year     16737 non-null  int64  
 13  Enrollment Month    16737 non-null  int64  
 14  Cancellation Year   2067 non-null   float64
 15  Cancellation Month  2067 non-null   float64
dtypes: f

In [39]:
df_loyalty['Salary'].isnull().sum()/ df_loyalty.shape[0]*100

25.321144769074504

In [40]:

df_loyalty['Salary'].isnull().sum()

4238

In [41]:
df_loyalty[df_loyalty['Salary'].isnull()].sample(10)

,Loyalty Number,Country,Province,City,Postal Code,Gender,Education,Salary,Marital Status,Loyalty Card,CLV,Enrollment Type,Enrollment Year,Enrollment Month,Cancellation Year,Cancellation Month
9523,395622,Canada,British Columbia,Vancouver,V6E 3D9,Male,College,NaN,Married,Nova,21112.54,Standard,2015,6,NaN,NaN
10589,511568,Canada,Ontario,Toronto,P2T 6G3,Male,College,NaN,Married,Star,2460.67,Standard,2015,5,NaN,NaN
14399,392619,Canada,Ontario,Toronto,P2T 6G3,Male,College,NaN,Single,Star,6993.84,Standard,2014,2,NaN,NaN
6157,713580,Canada,Ontario,Toronto,M2P 4F6,Male,College,NaN,Single,Nova,4853.53,Standard,2017,7,NaN,NaN
875,894340,Canada,Prince Edward Island,Charlottetown,C1A 6E8,Male,College,NaN,Single,Aurora,5111.36,Standard,2013,4,NaN,NaN
477,367961,Canada,Ontario,Ottawa,K1F 2R2,Male,College,NaN,Divorced,Star,4670.95,Standard,2013,6,NaN,NaN
12270,148453,Canada,British Columbia,Vancouver,V5R 1W3,Female,College,NaN,Single,Star,3479.14,Standard,2018,7,NaN,NaN
6449,893555,Canada,British Columbia,Vancouver,V5R 1W3,Female,College,NaN,Single,Nova,5179.55,Standard,2016,5,2017.0,1.0
9925,322193,Canada,British Columbia,Vancouver,V5R 1W3,Male,College,NaN,Single,Star,2175.62,Standard,2016,4,NaN,NaN
3297,721973,Canada,Ontario,London,M5B 3E4,Female,College,NaN,Single,Aurora,11905.07,Standard,2014,9,NaN,NaN


In [42]:
df_loyalty[df_loyalty['Salary'] < 0]

,Loyalty Number,Country,Province,City,Postal Code,Gender,Education,Salary,Marital Status,Loyalty Card,CLV,Enrollment Type,Enrollment Year,Enrollment Month,Cancellation Year,Cancellation Month
1082,542976,Canada,Quebec,Montreal,H2Y 4R4,Male,High School or Below,-49830.0,Divorced,Star,24127.50,2018 Promotion,2018,3,NaN,NaN
1894,959977,Canada,British Columbia,Vancouver,V5R 1W3,Female,Bachelor,-12497.0,Married,Aurora,9453.00,2018 Promotion,2018,3,NaN,NaN
2471,232755,Canada,British Columbia,Vancouver,V1E 4R6,Female,Bachelor,-46683.0,Single,Nova,4787.81,2018 Promotion,2018,3,NaN,NaN
3575,525245,Canada,British Columbia,Victoria,V10 6T5,Male,Bachelor,-45962.0,Married,Star,2402.33,2018 Promotion,2018,3,NaN,NaN
3932,603070,Canada,British Columbia,West Vancouver,V6V 8Z3,Female,Bachelor,-19325.0,Single,Star,2893.74,2018 Promotion,2018,3,NaN,NaN
4712,491242,Canada,British Columbia,Dawson Creek,U5I 4F1,Male,Bachelor,-43234.0,Married,Star,7597.91,2018 Promotion,2018,3,NaN,NaN
6560,115505,Canada,Newfoundland,St. John's,A1C 6H9,Male,Bachelor,-10605.0,Married,Nova,5860.17,2018 Promotion,2018,4,NaN,NaN
6570,430398,Canada,Newfoundland,St. John's,A1C 6H9,Male,Bachelor,-17534.0,Married,Nova,49423.80,2018 Promotion,2018,3,NaN,NaN
7373,152016,Canada,Ontario,Toronto,P1J 8T7,Female,Bachelor,-58486.0,Married,Aurora,5067.21,2018 Promotion,2018,2,NaN,NaN
8576,194065,Canada,Ontario,Sudbury,M5V 1G5,Female,Bachelor,-31911.0,Married,Nova,2888.85,2018 Promotion,2018,2,NaN,NaN
